In [1]:
# Install, import libraries
import sys
!{sys.executable} -m pip install google-api-python-client earthengine-api

# Import, set up Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/xAF8EogQfOrMPweZ8cU2F0isJKT2xKNFotzpU0Tbzq_RSCDZWCo5LlU



Successfully saved authorization token.


In [2]:
import tensorflow as tf

tf.enable_eager_execution()
print(tf.__version__)

import folium
print(folium.__version__)

/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/frame

1.13.1
0.10.1


In [3]:
shapefile = ee.FeatureCollection('users/pandringa/gmw_2009');

costa_rica = shapefile.filterBounds(ee.Geometry.Rectangle(-82.5, 8, -81.5, 9)) # Mangrove forest: 8.235541, -82.373174
print('Mangrove shapes in Costa Rica\'s 1x1 square:', len(costa_rica.getInfo()['features']))

chapel_hill_polygon = ee.Geometry.Polygon([
    [-79.0899192833252, 35.928421884080535],
    [-79.02846451037598, 35.92897789378388],
    [-79.02777786486817, 35.891716600077054],
    [-79.08923263781739, 35.89227286777993],
])
chapel_hill = shapefile.filterBounds(chapel_hill_polygon)
print('Mangrove shapes in Chapel Hill:', len(chapel_hill.getInfo()['features']))

Mangrove shapes in Costa Rica's 1x1 square: 651
Mangrove shapes in Chapel Hill: 0


In [4]:
costa_rica.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-82.39200073005266, 8.33777879112791],
    [-82.3917777760015, 8.337778748743364],
    [-82.3917777760015, 8.337109932828263],
    [-82.39155482123647, 8.337109890596109],
    [-82.39133186575759, 8.337109931170067],
    [-82.39133186575759, 8.338001713937947],
    [-82.39155482123647, 8.338001673755839],
    [-82.39155482123647, 8.338220245640683],
    [-82.3917777760015, 8.33822020541231],
    [-82.3917777760015, 8.338001716380825],
    [-82.39200073005266, 8.33800167591415],
    [-82.39200073005266, 8.33777879112791]]]},
 'id': '000d00000000000004ff',
 'properties': {'ogc_fid': 469055, 'pxlval': 1}}

In [8]:
# Change the following two lines to use your own training data.
labels = ee.FeatureCollection('users/pandringa/gmw_2009');
label = 'plxval'

l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1')

# The image input data is a 2018 cloud-masked median composite.
image = l7.filterDate('2018-01-01', '2018-12-31').median()


# Sample the image at the points and add a random column.
sample = image.sampleRegions(
  collection=labels, properties=[label], scale=30).randomColumn()

# Partition the sample approximately 70-30.
training = sample.filter(ee.Filter.lt('random', 0.7))
testing = sample.filter(ee.Filter.gte('random', 0.7))


from pprint import pprint

# Print the first couple points to verify.
pprint({'training': training.first().getInfo()})
pprint({'testing': testing.first().getInfo()})

KeyboardInterrupt: 

Following Cells from https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb#scrollTo=uNc7a2nRR4MI

In [11]:
landsat8_collection = ee.ImageCollection('LANDSAT/LE07/C01/T1').filterDate('2016-01-01', '2018-04-19').min()
landsat8_collection = landsat8_collection.slice(0,9)

vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1],
    'region':shapefile} 

image = landsat8_collection.clip(shapefile)


mapid = image.getMapId(vis)

map = folium.Map(location=[19.15,-98.75],zoom_start=9, height=500,width=700)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Landsat 8 ',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

In [19]:
def NDVI(image):
    return image.normalizedDifference(['B5', 'B4'])

def SAM(image):
    band1 = image.select("B1")
    bandn = image.select("B2", "B3", "B4", "B5");
    maxObjSize = 256;
    b = band1.divide(bandn);
    spectralAngleMap = b.atan();
    spectralAngleMap_sin = spectralAngleMap.sin();
    spectralAngleMap_cos = spectralAngleMap.cos();
    sum_cos = spectralAngleMap_cos.reduce(ee.call("Reducer.sum"));
    sum_sin = spectralAngleMap_sin.reduce(ee.call("Reducer.sum"));
    return ee.Image.cat(sum_sin, sum_cos, spectralAngleMap_sin, spectralAngleMap_cos);

#Enhanced Vegetation Index
def EVI(image):
    # L(Canopy background)
    # C1,C2(Coefficients of aerosol resistance term)
    # GainFactor(Gain or scaling factor)
    gain_factor = ee.Image(2.5);
    coefficient_1 = ee.Image(6);
    coefficient_2 = ee.Image(7.5);
    l = ee.Image(1);
    nir = image.select("B5");
    red = image.select("B4");
    blue = image.select("B2");
    evi = image.expression(
        "Gain_Factor*((NIR-RED)/(NIR+C1*RED-C2*BLUE+L))",
        {
            "Gain_Factor":gain_factor,
            "NIR":nir,
            "RED":red,
            "C1":coefficient_1,
            "C2":coefficient_2,
            "BLUE":blue,
            "L":l
        }
    )
    return evi

#Atmospherically Resistant Vegetation Index
def ARVI(image):
    red = image.select("B4")
    blue = image.select("B2")
    nir = image.select("B5")
    red_square = red.multiply(red)
    arvi = image.expression(
        "NIR - (REDsq - BLUE)/(NIR+(REDsq-BLUE))",{
            "NIR": nir,
            "REDsq": red_square,
            "BLUE": blue
        }
    )
    return arvi

#Leaf Area Index
def LAI(image):
    nir = image.select("B5")
    red = image.select("B4")
    coeff1 = ee.Image(0.0305);
    coeff2 = ee.Image(1.2640);
    lai = image.expression(
        "(((NIR/RED)*COEFF1)+COEFF2)",
        {
            "NIR":nir,
            "RED":red,
            "COEFF1":coeff1,
            "COEFF2":coeff2
        }
    )
    return lai

def tasseled_cap_transformation(image):
    #Tasseled Cap Transformation for Landsat 8 based on the 
    #scientfic work "Derivation of a tasselled cap transformation based on Landsat 8 at-satellite reflectance" 
    #by M.Baigab, L.Zhang, T.Shuai & Q.Tong (2014). The bands of the output image are the brightness index, 
    #greenness index and wetness index.
    b = image.select("B2", "B3", "B4", "B5");
    #Coefficients are only for Landsat 8 TOA
    brightness_coefficents= ee.Image([0.3029, 0.2786, 0.4733, 0.5599, 0.508, 0.1872])
    greenness_coefficents= ee.Image([-0.2941, -0.243, -0.5424, 0.7276, 0.0713, -0.1608]);
    wetness_coefficents= ee.Image([0.1511, 0.1973, 0.3283, 0.3407, -0.7117, -0.4559]);
    fourth_coefficents= ee.Image([-0.8239, 0.0849, 0.4396, -0.058, 0.2013, -0.2773]);
    fifth_coefficents= ee.Image([-0.3294, 0.0557, 0.1056, 0.1855, -0.4349, 0.8085]);
    sixth_coefficents= ee.Image([0.1079, -0.9023, 0.4119, 0.0575, -0.0259, 0.0252]);
    
    #Calculate tasseled cap transformation
    brightness = image.expression(
        '(B * BRIGHTNESS)',
        {
            'B':b,
            'BRIGHTNESS': brightness_coefficents
        })
    greenness = image.expression(
        '(B * GREENNESS)',
        {
            'B':b,
            'GREENNESS': greenness_coefficents
        })
    wetness = image.expression(
        '(B * WETNESS)',
        {
            'B':b,
            'WETNESS': wetness_coefficents
        })
    fourth = image.expression(
        '(B * FOURTH)',
        {
            'B':b,
            'FOURTH': fourth_coefficents
        })
    fifth = image.expression(
        '(B * FIFTH)',
        {
            'B':b,
            'FIFTH': fifth_coefficents
        })
    sixth = image.expression(
        '(B * SIXTH)',
        {
            'B':b,
            'SIXTH': sixth_coefficents
        })
    bright = brightness.reduce(ee.call("Reducer.sum"));
    green = greenness.reduce(ee.call("Reducer.sum"));
    wet = wetness.reduce(ee.call("Reducer.sum"));
    four = fourth.reduce(ee.call("Reducer.sum"));
    five = fifth.reduce(ee.call("Reducer.sum"));
    six = sixth.reduce(ee.call("Reducer.sum"));
    tasseled_cap = ee.Image(bright).addBands(green).addBands(wet).addBands(four).addBands(five).addBands(six)
    return tasseled_cap.rename('brightness','greenness','wetness','fourth','fifth','sixth')

tct = tasseled_cap_transformation(landsat8_collection)

In [23]:
image = tct.clip(shapefile)

vis_tct = {'min':-1,'max':2,'size':'800',
          'bands':["B2", "B3", "B4", "B5"],
          'region':shapefile}


mapid = image.getMapId(vis_tct)

map = folium.Map(location=[19.15,-98.75],zoom_start=9, height=500,width=700)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Tasseled Cap Transformation',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

EEException: Image.multiply: Images must contain the same number of bands or only 1 band. Got 4 and 6.

In [14]:

ndvi = NDVI(landsat8_collection)
sam = SAM(landsat8_collection)
evi = EVI(landsat8_collection)
arvi = ARVI(landsat8_collection)
lai = LAI(landsat8_collection)
spectral_indices_stack = ee.Image(ndvi).addBands(lai).addBands(sam).addBands(arvi).addBands(evi).addBands(tct).addBands(landsat8_collection)

image = ndvi.clip(shapefile)

vis_ndvi = {'min':-1,'max':1,'size':'800',
           'region':shapefile}

mapid = image.getMapId(vis_ndvi)

map = folium.Map(location=[19.15,-98.75],zoom_start=9, height=500,width=700)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NDVI',
  ).add_to(map)

map.add_child(folium.LayerControl())
map



NameError: name 'tct' is not defined